In [1]:
import cv2
import os
import numpy as np
import pandas as pd
import requests
from rembg import remove
from tqdm import tqdm
%matplotlib inline
import matplotlib.pyplot as plt


In [93]:
df = pd.read_csv('OpenData_PotOpenTabletIdntfcC20220601.csv')
dir = 'image_bg_removed'

In [67]:
def subimage(image, rect):
    theta = rect[2]-90
    center = (int(rect[0][0]),int(rect[0][1]))
    height = int(rect[1][0])
    width = int(rect[1][1])
    theta *= 3.14159 / 180 # convert to rad
    v_x = (np.cos(theta), np.sin(theta))
    v_y = (-np.sin(theta), np.cos(theta))
    s_x = center[0] - v_x[0] * ((width-1) / 2) - v_y[0] * ((height-1) / 2)
    s_y = center[1] - v_x[1] * ((width-1) / 2) - v_y[1] * ((height-1) / 2)

    mapping = np.array([[v_x[0],v_y[0], s_x],
                        [v_x[1],v_y[1], s_y]])

    return cv2.warpAffine(image,mapping,(width, height),flags=cv2.WARP_INVERSE_MAP,borderMode=cv2.BORDER_REPLICATE)

In [94]:
success = os.listdir('image_normalized')

In [95]:
for i ,row in tqdm(df.iterrows()):
        index = row['품목일련번호']
        if str(index) +'.png' in success:
                continue
        img = cv2.imread(os.path.join('image_bg_removed',str(index) +'.png'))
        imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, th = cv2.threshold(imgray, 1,255,cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        pills = []
        inds = []
        counts = 0
        for j,contr in enumerate(contours):
                rect = cv2.boundingRect(contr)
                if rect[3]*rect[2] > 1000:
                        counts += 1
                        inds.append(j)
        if len(inds) == 2:
                for ind in inds:
                        if row['의약품제형'] == '원형':
                                rect = cv2.boundingRect(contours[ind])
                                rect = ((rect[0]+rect[3]//2,rect[1]+rect[2]//2),(rect[3],rect[2]),90)
                        else:
                                rect = cv2.minAreaRect(contours[ind])
                        crop = subimage(img,rect)
                        if crop.shape[0] > crop.shape[1]:
                                crop = cv2.rotate(crop, cv2.ROTATE_90_CLOCKWISE)
                        crop = cv2.resize(crop,(256,int(256*crop.shape[0]/crop.shape[1])))
                        pills.append(crop)
                normalized = np.zeros((512,512,3),dtype=np.uint8)
                normalized[128-pills[0].shape[0]//2 : 128 +(pills[0].shape[0]-pills[0].shape[0]//2),128-pills[0].shape[1]//2 : 128 +(pills[0].shape[1]-pills[0].shape[1]//2)] = pills[0]
                normalized[384-pills[1].shape[0]//2 : 384 +(pills[1].shape[0]-pills[1].shape[0]//2),384-pills[1].shape[1]//2 : 384 +(pills[1].shape[1]-pills[1].shape[1]//2)] = pills[1]
                cv2.imwrite(os.path.join('success5',str(index) +'.png'),normalized)
        else:
                print(i)
                cv2.imwrite(os.path.join('fail5',str(index) +'.png'),img)
        

24372it [00:06, 3635.41it/s]


In [ ]:
for i ,row in tqdm(df.iterrows()):
        index = row['품목일련번호']
        
        img = cv2.imread(os.path.join('image_bg_removed',str(index) +'.png'))
        imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, th = cv2.threshold(imgray, 10,255,cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        pills = []
        counts = 0
        inds = []
        for j,contr in enumerate(contours):
                rect = cv2.boundingRect(contr)
                if rect[3]*rect[2] > 1000:
                        counts += 1
                        inds.append(j)
        if len(inds) == 2:
                for ind in inds:
                        if row['의약품제형'] == '원형':
                                rect = cv2.boundingRect(contours[ind])
                                rect = ((rect[0]+rect[3]//2,rect[1]+rect[2]//2),(rect[3],rect[2]),90)
                        else:
                                rect = cv2.minAreaRect(contr)
                        crop = subimage(img,rect)
                        if crop.shape[0] > crop.shape[1]:
                                crop = cv2.rotate(crop, cv2.ROTATE_90_CLOCKWISE)
                        crop = cv2.resize(crop,(256,int(256*crop.shape[0]/crop.shape[1])))
                        pills.append(crop)
                normalized = np.zeros((512,512,3),dtype=np.uint8)
                normalized[128-pills[0].shape[0]//2 : 128 +(pills[0].shape[0]-pills[0].shape[0]//2),128-pills[0].shape[1]//2 : 128 +(pills[0].shape[1]-pills[0].shape[1]//2)] = pills[0]
                normalized[384-pills[1].shape[0]//2 : 384 +(pills[1].shape[0]-pills[1].shape[0]//2),384-pills[1].shape[1]//2 : 384 +(pills[1].shape[1]-pills[1].shape[1]//2)] = pills[1]
                cv2.imwrite(os.path.join('image_normalized',str(index) +'.png'),normalized)
        else:
                print(i)
                cv2.imwrite(os.path.join('image_failed',str(index) +'.png'),img)

In [64]:
img = cv2.cvtColor(cv2.imread('fail4/199202070.png'),cv2.COLOR_BGR2RGB)
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, th = cv2.threshold(imgray, 60,255,cv2.THRESH_BINARY)

# 컨튜어 찾기
contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
print(len(contours))
contr = contours[0]
rect = cv2.minAreaRect(contr)
print(rect)
box = cv2.boxPoints(rect)   # 중심점과 각도를 4개의 꼭지점 좌표로 변환
box = np.int0(box)          # 정수로 변환
cv2.drawContours(img, [box], -1, (0,255,0), 3)
plt.imshow(th)

[ WARN:0@3083.398] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('fail4/199202070.png'): can't open/read file: check file path/integrity


error: OpenCV(4.5.5) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [38]:
len(os.listdir('image_normalized'))

24295

In [65]:
len(os.listdir('fail5'))

46